In [1]:
import sklearn
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor

In [3]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

In [4]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib

In [5]:
dataset_url = 'http://mlr.cs.umass.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
data = pd.read_csv(dataset_url)

In [8]:
data = pd.read_csv(dataset_url, sep=';')
print(data.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5 

In [9]:
print(data.shape)

(1599, 12)


In [11]:
print(data.describe())

       fixed acidity  volatile acidity  citric acid  residual sugar  \
count    1599.000000       1599.000000  1599.000000     1599.000000   
mean        8.319637          0.527821     0.270976        2.538806   
std         1.741096          0.179060     0.194801        1.409928   
min         4.600000          0.120000     0.000000        0.900000   
25%         7.100000          0.390000     0.090000        1.900000   
50%         7.900000          0.520000     0.260000        2.200000   
75%         9.200000          0.640000     0.420000        2.600000   
max        15.900000          1.580000     1.000000       15.500000   

         chlorides  free sulfur dioxide  total sulfur dioxide      density  \
count  1599.000000          1599.000000           1599.000000  1599.000000   
mean      0.087467            15.874922             46.467792     0.996747   
std       0.047065            10.460157             32.895324     0.001887   
min       0.012000             1.000000         

In [12]:
y = data.quality
X = data.drop('quality', axis=1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123, stratify=y)

In [14]:
scaler = preprocessing.StandardScaler().fit(X_train)

In [15]:
X_train_scaled = scaler.transform(X_train)
print(X_train_scaled.mean(axis=0))

[  1.16664562e-16  -3.05550043e-17  -8.47206937e-17  -2.22218213e-17
   2.22218213e-17  -6.38877362e-17  -4.16659149e-18  -2.54439854e-15
  -8.70817622e-16  -4.08325966e-16  -1.17220107e-15]


In [16]:
print(X_train_scaled.std(axis=0))

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


In [17]:
X_test_scaled = scaler.transform(X_test)
print(X_test_scaled.mean(axis=0))

[ 0.02776704  0.02592492 -0.03078587 -0.03137977 -0.00471876 -0.04413827
 -0.02414174 -0.00293273 -0.00467444 -0.10894663  0.01043391]


In [19]:
print(X_test_scaled.std(axis=0))

[ 1.02160495  1.00135689  0.97456598  0.91099054  0.86716698  0.94193125
  1.03673213  1.03145119  0.95734849  0.83829505  1.0286218 ]


In [20]:
pipeline = make_pipeline(preprocessing.StandardScaler(), RandomForestRegressor(n_estimators=100))

In [22]:
print(pipeline.get_params())

{'steps': [('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False))], 'standardscaler': StandardScaler(copy=True, with_mean=True, with_std=True), 'randomforestregressor': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False), 'standardscaler__copy': True, 'standardscaler__with_mean': True,

In [26]:
hyperparameters = {'randomforestregressor__max_features': ['auto', 'sqrt', 'log2'], 'randomforestregressor__max_depth': [None, 5, 3, 1]}

In [27]:
clf = GridSearchCV(pipeline, hyperparameters, cv=10)
clf.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'randomforestregressor__max_features': ['auto', 'sqrt', 'log2'], 'randomforestregressor__max_depth': [None, 5, 3, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [28]:
print(clf.best_params_)

{'randomforestregressor__max_depth': None, 'randomforestregressor__max_features': 'auto'}


In [29]:
print(clf.refit)

True


In [31]:
y_pred = clf.predict(X_test)

In [32]:
print(r2_score(y_test, y_pred))

0.469327198983


In [33]:
print(mean_squared_error(y_test, y_pred))

0.3424290625


In [34]:
joblib.dump(clf, 'rf_regressor.pkl')

['rf_regressor.pkl']

In [36]:
clf2 = joblib.load('rf_regressor.pkl')

In [38]:
clf2.predict(X_test)

array([ 6.48,  5.7 ,  4.98,  5.6 ,  6.39,  5.63,  4.86,  4.65,  5.01,
        5.89,  5.28,  5.7 ,  5.89,  4.99,  5.83,  5.6 ,  6.74,  5.79,
        5.71,  6.96,  5.64,  5.7 ,  5.02,  6.02,  5.86,  5.03,  5.51,
        5.16,  5.96,  5.99,  5.93,  6.54,  5.99,  4.99,  4.91,  5.97,
        5.  ,  5.79,  5.02,  5.91,  4.86,  5.92,  6.67,  5.1 ,  6.26,
        5.38,  5.63,  5.47,  5.07,  6.58,  5.89,  5.15,  5.87,  5.13,
        5.57,  5.83,  5.18,  5.46,  4.96,  5.35,  5.27,  5.14,  5.02,
        5.83,  5.97,  5.33,  6.33,  5.03,  5.08,  6.77,  5.78,  5.48,
        5.05,  5.04,  5.3 ,  6.  ,  5.33,  5.1 ,  5.1 ,  5.26,  6.67,
        5.57,  6.33,  6.66,  5.07,  5.91,  6.59,  6.15,  5.64,  5.73,
        5.94,  5.45,  6.42,  5.7 ,  5.73,  5.79,  6.71,  6.78,  5.47,
        6.87,  5.11,  5.28,  5.11,  6.68,  4.97,  4.64,  5.71,  5.06,
        5.76,  5.97,  5.79,  5.37,  6.09,  5.47,  5.04,  5.24,  5.87,
        5.04,  4.87,  5.91,  5.84,  5.06,  5.88,  5.96,  5.26,  5.26,
        5.19,  5.91,